In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

/kaggle/input/housesalesprediction/kc_house_data.csv


In [2]:
data = pd.read_csv("../input/housesalesprediction/kc_house_data.csv")
data

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [4]:
data = data.drop('id',axis =1)

In [5]:
data['year'] = data['date'].apply(lambda x: x[0:4]) # get year 
data['month'] = data['date'].apply(lambda x: x[4:6]) 
data = data.drop('date',axis =1)
data

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,...,0,1955,0,98178,47.5112,-122.257,1340,5650,2014,10
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,...,400,1951,1991,98125,47.7210,-122.319,1690,7639,2014,12
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,...,0,1933,0,98028,47.7379,-122.233,2720,8062,2015,02
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,...,910,1965,0,98136,47.5208,-122.393,1360,5000,2014,12
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,...,0,1987,0,98074,47.6168,-122.045,1800,7503,2015,02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,...,0,2009,0,98103,47.6993,-122.346,1530,1509,2014,05
21609,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,...,0,2014,0,98146,47.5107,-122.362,1830,7200,2015,02
21610,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,...,0,2009,0,98144,47.5944,-122.299,1020,2007,2014,06
21611,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,...,0,2004,0,98027,47.5345,-122.069,1410,1287,2015,01


In [6]:
len(data['zipcode'].unique()) # 1 hot encode areas

70

In [7]:
def oneHot_encoding(df, column, prefix ):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix = prefix)
    df =  pd.concat([df,dummies],axis =1)
    return df.drop(column,axis = 1)

In [8]:
data = oneHot_encoding(data,'zipcode', 'zip')
data

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,...,0,0,0,0,0,0,1,0,0,0
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,0
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,...,0,0,0,0,0,0,0,0,0,0
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,...,0,0,0,0,0,0,0,0,0,0
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
21609,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,...,1,0,0,0,0,0,0,0,0,0
21610,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,0
21611,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0


In [9]:
data.query('yr_renovated != 0')

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,0
35,696000.0,3,2.50,2300,3060,1.5,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
95,905000.0,4,2.50,3300,10250,1.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,0
103,1088000.0,3,2.50,2920,8113,2.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
115,740500.0,3,3.50,4380,6350,2.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19622,1150000.0,3,1.75,1760,6788,2.0,1,4,3,7,...,0,0,0,0,0,0,0,0,0,0
20057,434900.0,3,2.00,1520,5040,2.0,0,0,3,7,...,0,0,0,0,0,1,0,0,0,0
20444,500012.0,4,2.50,2400,9612,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
20447,356999.0,3,1.50,1010,1546,2.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0


In [10]:
data.query('yr_renovated == 0')

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,...,0,0,0,0,0,0,1,0,0,0
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,...,0,0,0,0,0,0,0,0,0,0
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,...,0,0,0,0,0,0,0,0,0,0
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
5,1225000.0,4,4.50,5420,101930,1.0,0,0,3,11,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
21609,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,...,1,0,0,0,0,0,0,0,0,0
21610,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,0
21611,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0


In [11]:
data = data.drop('yr_renovated', axis = 1)
data

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,...,0,0,0,0,0,0,1,0,0,0
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,0
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,...,0,0,0,0,0,0,0,0,0,0
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,...,0,0,0,0,0,0,0,0,0,0
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
21609,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,...,1,0,0,0,0,0,0,0,0,0
21610,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,0
21611,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0


split and scale 

In [12]:
y = data['price'].copy() # y: labels, x: features
x =  data.drop('price',axis = 1).copy()

In [13]:
scaler =  StandardScaler()
x = scaler.fit_transform(x)

In [14]:
tf_x_train,tf_x_test,tf_y_train,tf_y_test = train_test_split(x,y,train_size=0.7,random_state = 1) # 

train w tensorflow

In [15]:
inputs =  tf.keras.Input(shape=(88,))
hidden = tf.keras.layers.Dense(64, activation='relu')(inputs)
hidden = tf.keras.layers.Dense(64,activation='relu')(hidden)
outputs =  tf.keras.layers.Dense(1,activation = 'linear')(hidden)

tf_model = tf.keras.Model(inputs,outputs)

tf_model.compile( optimizer = 'adam', loss = 'mse')

history = tf_model.fit(
    tf_x_train, tf_y_train, validation_split=0.12, batch_size =32, epochs = 10
)


Epoch 1/10
417/417 [==============================] - 2s 3ms/step - loss: 408594278586.1818 - val_loss: 440905433088.0000
Epoch 2/10
417/417 [==============================] - 1s 2ms/step - loss: 407102742410.4115 - val_loss: 390954909696.0000
Epoch 3/10
417/417 [==============================] - 1s 1ms/step - loss: 335160512178.8325 - val_loss: 290310914048.0000
Epoch 4/10
417/417 [==============================] - 1s 1ms/step - loss: 237886420453.0526 - val_loss: 187678490624.0000
Epoch 5/10
417/417 [==============================] - 1s 1ms/step - loss: 156661418405.3589 - val_loss: 126150418432.0000
Epoch 6/10
417/417 [==============================] - 1s 1ms/step - loss: 106311572406.5072 - val_loss: 97369112576.0000
Epoch 7/10
417/417 [==============================] - 1s 2ms/step - loss: 84266292689.4545 - val_loss: 80159481856.0000
Epoch 8/10
417/417 [==============================] - 1s 2ms/step - loss: 66169556560.8421 - val_loss: 67083325440.0000
Epoch 9/10
417/417 [=========

In [16]:
tf_rmse = np.sqrt(tf_model.evaluate(tf_x_test,tf_y_test))

203/203 [==============================] - 0s 913us/step - loss: 54417764352.0000


training w pytorch

In [17]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(88, 64)
        self.layer2 = nn.Linear(64, 64)
        self.out = nn.Linear(64, 1)
    
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.out(x)
        return x

net = Net()

In [18]:
list(net.parameters())

[Parameter containing:
 tensor([[-0.0110,  0.0796,  0.0914,  ...,  0.0129,  0.0763,  0.0433],
         [-0.0105, -0.0259,  0.0391,  ...,  0.0682,  0.0394, -0.0507],
         [ 0.0068,  0.0552,  0.0232,  ..., -0.0896, -0.0515,  0.1029],
         ...,
         [-0.0359,  0.1039,  0.0155,  ...,  0.0775,  0.0642,  0.0936],
         [-0.0748,  0.0708, -0.0050,  ...,  0.0105, -0.0881,  0.0838],
         [ 0.0784,  0.1032,  0.0001,  ...,  0.0568, -0.0063, -0.0976]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0103, -0.0899,  0.0396,  0.0037,  0.0899,  0.0696,  0.0831, -0.0955,
         -0.0293,  0.0280,  0.0805, -0.0493, -0.1060,  0.0456, -0.0948,  0.0456,
         -0.0804,  0.0677,  0.0526,  0.0692, -0.0849,  0.0913,  0.0922,  0.0608,
         -0.0094, -0.0517,  0.0759,  0.0745,  0.0323, -0.0840, -0.0036,  0.0138,
          0.0244, -0.0503, -0.0014,  0.0384, -0.0370, -0.0138, -0.1049,  0.0472,
         -0.0137,  0.0759, -0.0562,  0.0876,  0.0103,  0.0851,  0.0712,  0.0530

In [19]:
for i in range(len(list(net.parameters()))):
    print(list(net.parameters())[i].shape) # in and out of layers 

torch.Size([64, 88])
torch.Size([64])
torch.Size([64, 64])
torch.Size([64])
torch.Size([1, 64])
torch.Size([1])


In [20]:
# need to convert for torch: tf_x_train,tf_x_test,tf_y_train,tf_y_test

torch_x_train = torch.tensor(tf_x_train).type(torch.float32)
torch_y_train = torch.tensor(np.array(tf_y_train)).type(torch.float32)

torch_x_test = torch.tensor(tf_x_test).type(torch.float32)
torch_y_test = torch.tensor(np.array(tf_y_test)).type(torch.float32)

In [21]:
optimizer = torch.optim.Adam(net.parameters(), lr = 0.01)
criterion = nn.MSELoss()

In [22]:
for x,target in zip(torch_x_train, torch_y_train): # train
    optimizer.zero_grad()
    output = net(x)
    loss = criterion(output,target)
    loss.backward()
    optimizer.step()
 

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [23]:
total_loss = 0
#test
for x, target in zip(torch_x_test, torch_y_test):
    output = net(x)
    loss = criterion(output, target)
    total_loss += loss
    
avg_loss = total_loss / len(torch_x_test)
avg_loss

tensor(4.5964e+10, grad_fn=<DivBackward0>)

In [24]:
torch_rmse = torch.sqrt(avg_loss).detach().numpy()
torch_rmse

array(214393.33, dtype=float32)

results torch vs tensor 

In [25]:
print("TensorFlow RMSE:", tf_rmse)
print("PyTorch RMSE:", torch_rmse)

TensorFlow RMSE: 233276.15470081806
PyTorch RMSE: 214393.33
